# Join and Pre-Process the New York City Datasets

Code responsible to Join and Pre-Process three New York City datasets:

- Airbnb: This data will be used to evaluate the mean price of housing in each borough
- Housing: Info about Projects about each Bourough with the number of Poor and Rich house units
- Census: Census data of New York, with important info about: Race, Income, Poverty, Child Poverty and other sensible infos

In [1]:
import pandas as pd

In [4]:
df_airbnb = pd.read_table("Input/airbnb_newyork.csv", sep = ',')
df_housing = pd.read_table("Input/housing_newyork.csv", sep = ',')
df_census = pd.read_table("Input/census_newyork.csv", sep = ',')
df_crimes = pd.read_table("Input/crimes_newyork.csv", sep = ',')

/usr/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [26]:
dict_county = {'BRONX': 'BRONX', 'BROOKLYN': 'KINGS', 'MANHATTAN': 'NEW YORK',
               'QUEENS': 'QUEENS', 'STATEN ISLAND': 'RICHMOND'}

In [5]:
df_crimes.shape

(1048575, 24)

### AirBNB: Removing NoN Interesting Columns

In [28]:
removed_columns = ['id', 'name', 'host_id', 'host_name', 'latitude', 'longitude',
                   'room_type', 'last_review', 'neighbourhood', 'calculated_host_listings_count',
                   'neighbourhood_group']
df_airbnb.fillna(value=0, axis=1, inplace=True)
df_airbnb['neighbourhood_group'] = list(map(lambda x: x.upper(), df_airbnb['neighbourhood_group']))
df_airbnb['County'] = list(map(lambda x: dict_county[x], df_airbnb['neighbourhood_group']))
df_airbnb = df_airbnb.drop(removed_columns, axis=1)
df_airbnb.head(5)

,price,minimum_nights,number_of_reviews,reviews_per_month,availability_365,County
0,149,1,9,0.21,365,KINGS
1,225,1,45,0.38,355,NEW YORK
2,150,3,0,0.00,365,NEW YORK
3,89,1,270,4.64,194,KINGS
4,80,10,9,0.10,0,NEW YORK


### AirBNB: Summarizing AirBNB per Borough

- Mean and Standard Deviation of each Numerical Row

In [29]:
token = 'County'
columns = list(df_airbnb.columns)
columns.remove(token)
mean_columns = list(map(lambda x: x + ' mean', columns))
std_columns = list(map(lambda x: x + ' std', columns))
df_summarized_airbnb = pd.DataFrame(columns = mean_columns + std_columns)

for index, borough in enumerate(set(df_airbnb[token])):

    df_aux = df_airbnb.loc[df_airbnb[token] == borough]

    for column in columns:
        df_summarized_airbnb.loc[index, column + ' mean'] =  df_aux.describe()[column]['mean']
        df_summarized_airbnb.loc[index, column + ' std'] =  df_aux.describe()[column]['std']
    
    df_summarized_airbnb.loc[index, 'Borough'] = borough

In [54]:
df_summarized_airbnb.head(5)

,price mean,minimum_nights mean,number_of_reviews mean,reviews_per_month mean,availability_365 mean,price std,minimum_nights std,number_of_reviews std,reviews_per_month std,availability_365 std,Borough
0,124.383,6.05656,24.2028,1.04979,100.232,186.874,17.6327,44.3449,1.45804,126.276,BROOKLYN
1,196.876,8.57915,20.9856,0.976782,111.979,291.383,24.0509,42.5723,1.52452,132.678,MANHATTAN
2,114.812,4.8311,30.941,1.57638,199.678,277.62,19.7276,44.8308,1.69071,131.852,STATEN ISLAND
3,87.4968,4.56095,26.0046,1.47566,165.759,106.709,15.6318,42.2148,1.6681,135.247,BRONX
4,99.5176,5.18143,27.7003,1.56708,144.452,167.102,15.0287,51.9559,2.13075,135.539,QUEENS


### Housing: Removing NoN Interesting Columns

In [30]:
pd.set_option('display.max_columns', 500)
removed_columns = ['Project ID', 'Project Start Date', 'Project Completion Date', 'Building ID',
                   'Number', 'Street', 'Postcode', 'Community Board', 'Council District',
                   'NTA - Neighborhood Tabulation Area', 'Latitude', 'Longitude', 'Project Name',
                   'Latitude (Internal)', 'Longitude (Internal)', 'Borough']

df_housing['Borough'] = list(map(lambda x: x.upper(), df_housing['Borough']))
df_housing['County'] = list(map(lambda x: dict_county[x], df_housing['Borough']))
df_housing = df_housing.drop(removed_columns, axis=1)
df_housing.head(5)

,BBL,BIN,Census Tract,Building Completion Date,Reporting Construction Type,Extended Affordability Only,Prevailing Wage Status,Extremely Low Income Units,Very Low Income Units,Low Income Units,Moderate Income Units,Middle Income Units,Other Income Units,Studio Units,1-BR Units,2-BR Units,3-BR Units,4-BR Units,5-BR Units,6-BR+ Units,Unknown-BR Units,Counted Rental Units,Counted Homeownership Units,All Counted Units,Total Units,County
0,1.019150e+09,NaN,226.0,NaN,New Construction,No,Non Prevailing Wage,0,16,24,0,38,1,16,25,32,6,0,0,0,0,79,0,79,79,NEW YORK
1,1.021170e+09,1062764.0,245.0,NaN,Preservation,No,Prevailing Wage,20,3,1,0,0,1,0,0,15,5,5,0,0,0,25,0,25,25,NEW YORK
2,1.021190e+09,1062843.0,245.0,NaN,Preservation,No,Prevailing Wage,6,11,3,0,0,0,0,0,20,0,0,0,0,0,20,0,20,20,NEW YORK
3,1.021190e+09,1062847.0,245.0,NaN,Preservation,No,Prevailing Wage,4,14,2,0,0,0,0,0,20,0,0,0,0,0,20,0,20,20,NEW YORK
4,NaN,NaN,NaN,NaN,New Construction,No,Non Prevailing Wage,1,18,22,8,0,1,11,13,14,12,0,0,0,0,50,0,50,50,KINGS


### Census: Removing NoN Interesting Columns

In [32]:
df_census.drop(['CensusTract', 'Borough'], axis=1)
df_census['County'] = list(map(lambda x: x.upper(), df_census['County']))
df_census.head(5)

,CensusTract,County,Borough,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Citizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,36005000100,BRONX,Bronx,7703,7133,570,29.9,6.1,60.9,0.2,1.6,6476,NaN,NaN,2440.0,373.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1,36005000200,BRONX,Bronx,5403,2659,2744,75.8,2.3,16.0,0.0,4.2,3639,72034.0,13991.0,22180.0,2206.0,20.0,20.7,28.7,17.1,23.9,8.0,22.3,44.8,13.7,38.6,2.9,0.0,0.0,43.0,2308,80.8,16.2,2.9,0.0,7.7
2,36005000400,BRONX,Bronx,5915,2896,3019,62.7,3.6,30.7,0.0,0.3,4100,74836.0,8407.0,27700.0,2449.0,13.2,23.6,32.2,23.4,24.9,9.0,10.5,41.3,10.0,44.6,1.4,0.5,2.1,45.0,2675,71.7,25.3,2.5,0.6,9.5
3,36005001600,BRONX,Bronx,5879,2558,3321,65.1,1.6,32.4,0.0,0.0,3536,32312.0,6859.0,17526.0,2945.0,26.3,35.9,19.1,36.1,26.2,4.9,13.8,37.2,5.3,45.5,8.6,1.6,1.7,38.8,2120,75.0,21.3,3.8,0.0,8.7
4,36005001900,BRONX,Bronx,2591,1206,1385,55.4,9.0,29.0,0.0,2.1,1557,37936.0,3771.0,17986.0,2692.0,37.1,31.5,35.4,20.9,26.2,6.6,11.0,19.2,5.3,63.9,3.0,2.4,6.2,45.4,1083,76.8,15.5,7.7,0.0,19.2


In [19]:
df_housing.head(5)

,Borough,BBL,BIN,Census Tract,Building Completion Date,Reporting Construction Type,Extended Affordability Only,Prevailing Wage Status,Extremely Low Income Units,Very Low Income Units,Low Income Units,Moderate Income Units,Middle Income Units,Other Income Units,Studio Units,1-BR Units,2-BR Units,3-BR Units,4-BR Units,5-BR Units,6-BR+ Units,Unknown-BR Units,Counted Rental Units,Counted Homeownership Units,All Counted Units,Total Units
0,Manhattan,1.019150e+09,NaN,226.0,NaN,New Construction,No,Non Prevailing Wage,0,16,24,0,38,1,16,25,32,6,0,0,0,0,79,0,79,79
1,Manhattan,1.021170e+09,1062764.0,245.0,NaN,Preservation,No,Prevailing Wage,20,3,1,0,0,1,0,0,15,5,5,0,0,0,25,0,25,25
2,Manhattan,1.021190e+09,1062843.0,245.0,NaN,Preservation,No,Prevailing Wage,6,11,3,0,0,0,0,0,20,0,0,0,0,0,20,0,20,20
3,Manhattan,1.021190e+09,1062847.0,245.0,NaN,Preservation,No,Prevailing Wage,4,14,2,0,0,0,0,0,20,0,0,0,0,0,20,0,20,20
4,Brooklyn,NaN,NaN,NaN,NaN,New Construction,No,Non Prevailing Wage,1,18,22,8,0,1,11,13,14,12,0,0,0,0,50,0,50,50


### Tasks:
- Verify the best way to complete the NaN values for each dataset
- Group the data by county
- Make a new dataset with the joined data
- 